# 06-05: WhatsApp Bot (Baileys)使用 Baileys 库连接 WhatsApp Web。

In [ ]:
// 安装: npm install @whiskeysockets/baileys
import makeWASocket, {
  DisconnectReason,
  useSingleFileAuthState,
  makeInMemoryStore
} from '@whiskeysockets/baileys';
import { Boom } from '@hapi/boom';
// ========== 1. 基础连接 ==========
async function connectToWhatsApp() {
  const { state, saveState } = useSingleFileAuthState('./auth_info.json');

  const sock = makeWASocket({
    auth: state,
    printQRInTerminal: true
  });

  sock.ev.on('connection.update', (update) => {
    const { connection, lastDisconnect, qr } = update;

    if (qr) {
      console.log('Scan QR code to connect');
    }

    if (connection === 'close') {
      const shouldReconnect =
        (lastDisconnect?.error as Boom)?.output?.statusCode !==
        DisconnectReason.loggedOut;

      console.log(
        'Connection closed due to: ',
        lastDisconnect?.error,
        ', reconnecting: ',
        shouldReconnect
      );

      if (shouldReconnect) {
        connectToWhatsApp();
      }
    } else if (connection === 'open') {
      console.log('Opened connection');
    }
  });

  sock.ev.on('creds.update', saveState);

  return sock;
}

In [ ]:
// ========== 2. 发送消息 ==========
async function sendMessages(sock: any) {
  // 发送文本消息
  await sock.sendMessage('1234567890@s.whatsapp.net', {
    text: 'Hello from Baileys!'
  });

  // 发送带提及的消息
  await sock.sendMessage('1234567890@s.whatsapp.net', {
    text: 'Hello @user!',
    mentions: ['1234567890@s.whatsapp.net']
  });

  // 发送回复
  await sock.sendMessage('1234567890@s.whatsapp.net', {
    text: 'This is a reply',
    quoted: originalMessage // 原消息对象
  });

  // 发送群消息
  await sock.sendMessage('1234567890@g.us', {
    text: 'Hello group!'
  });
}

In [ ]:
// ========== 3. 接收消息 ==========
function setupMessageHandler(sock: any) {
  sock.ev.on('messages.upsert', async (m: any) => {
    console.log(JSON.stringify(m, undefined, 2));

    const msg = m.messages[0];
    if (!msg.key.fromMe && m.type === 'notify') {
      console.log('Got message from:', msg.pushName);
      console.log('Message:', msg.message?.conversation);

      // 回复消息
      await sock.sendMessage(
        msg.key.remoteJid,
        { text: 'Hello there!' },
        { quoted: msg }
      );
    }
  });
}

In [ ]:
// ========== 4. 发送媒体 ==========
import { readFileSync } from 'fs';

async function sendMedia(sock: any, to: string) {
  // 发送图片
  await sock.sendMessage(to, {
    image: readFileSync('./image.png'),
    caption: 'Here is my image!'
  });

  // 发送视频
  await sock.sendMessage(to, {
    video: readFileSync('./video.mp4'),
    caption: 'Here is my video!',
    gifPlayback: false
  });

  // 发送音频
  await sock.sendMessage(to, {
    audio: readFileSync('./audio.mp3'),
    mimetype: 'audio/mp4',
    ptt: true // 语音消息
  });

  // 发送文档
  await sock.sendMessage(to, {
    document: readFileSync('./doc.pdf'),
    mimetype: 'application/pdf',
    fileName: 'document.pdf'
  });

  // 发送位置
  await sock.sendMessage(to, {
    location: {
      degreesLatitude: 24.121231,
      degreesLongitude: 55.1121221
    }
  });
}

In [ ]:
// ========== 5. 群组管理 ==========
async function manageGroups(sock: any) {
  // 获取所有群组
  const groups = await sock.groupFetchAllParticipating();
  console.log(Object.values(groups).map((g: any) => g.subject));

  // 创建群组
  const group = await sock.groupCreate('My New Group', [
    '1234567890@s.whatsapp.net',
    '0987654321@s.whatsapp.net'
  ]);
  console.log('Created group:', group.id);

  // 添加成员
  await sock.groupParticipantsUpdate(
    '1234567890@g.us',
    ['1111111111@s.whatsapp.net'],
    'add'
  );

  // 踢出成员
  await sock.groupParticipantsUpdate(
    '1234567890@g.us',
    ['1111111111@s.whatsapp.net'],
    'remove'
  );

  // 离开群组
  await sock.groupLeave('1234567890@g.us');
}

In [ ]:
// ========== 6. 状态和联系人 ==========
async function getInfo(sock: any) {
  // 获取状态
  const status = await sock.fetchStatus('1234567890@s.whatsapp.net');
  console.log('Status:', status);

  // 获取头像
  const profilePic = await sock.profilePictureUrl(
    '1234567890@s.whatsapp.net',
    'image'
  );

  // 检查用户是否存在
  const [result] = await sock.onWhatsApp('1234567890');
  console.log('Is on WhatsApp:', result?.exists);
}